Step 1 : Installing Dependecy

In [ ]:
!pip install transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 79.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.38.0 which is incompatible.


In [ ]:
!pip install pandas sentence-transformers faiss-cpu gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.0
    Uninstalling transformers-4.38.0:
      Successfully uninstalled transformers-4.38.0


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss
import numpy as np
import datasets
from datasets import load_dataset

Step 2: Import of Dataset

In [ ]:
# we will be use TruthfulQA as sample. Change to your dataset
ds = load_dataset("truthfulqa/truthful_qa", "generation")
df=ds['validation'].to_pandas()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/223k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

Step 3: Defining Functions

In [ ]:

# Setup Encoding Model
def encode_model():
  model = SentenceTransformer("all-MiniLM-L6-v2")
  return model

# Setup LLm Model
def chat_model():
  model_name = "Qwen/Qwen2.5-3B-Instruct" # change to any model you want
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model= AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
  return model,tokenizer


#Chat template for RAG
def generate_prompt(context,user_question: str) -> str:
    prompt = f"""Based on information below, give appropriate answer.Only give the answer.
Context:
{context}

Question: {user_question}
Answer:"""
    return prompt

# Creation of Vector Database (Using FAISS)
def vector_db():
  df['combined'] = df['question']+": " + df['best_answer']
  embeddings = model_encode.encode(df['combined'].tolist(), convert_to_numpy=True)
  dimension = embeddings.shape[1]
  index = faiss.IndexFlatL2(dimension)
  index.add(embeddings)
  return index

#  Vector Search query
def vector_search(que):
  query = que
  query_vec = model_encode.encode([query], convert_to_numpy=True)
  df['combined'] = df['question']+": " + df['best_answer']
  context = ""
  distances, relevant_indices = index.search(query_vec, k=1)
  for i in relevant_indices[0]:
    context += "\n" + df.loc[i, 'combined']

  return context

# Main System Function
def build_system2(user_question,history=[]):
  context=""
  context=vector_search(user_question)
  prompt = generate_prompt(context, user_question)
  inputs = tokenizer(prompt, return_tensors="pt").to(model_llm.device)
  output = model_llm.generate(
        **inputs,
        max_new_tokens=64,
        temperature=0.8,
        pad_token_id=tokenizer.eos_token_id,
    )

  response = tokenizer.decode(output[0], skip_special_tokens=True)
  response = response[len(prompt):].strip()
  res=str(response)
  history.append((user_question, res))
  context=""
  return history,history


Step 4: Initiazation of Vector Database and LLM model

In [ ]:
model_encode=encode_model()
index=vector_db()
model_llm,tokenizer=chat_model()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Step 5: Inferencing using Gradio UI

In [ ]:
import gradio as gr

def clear(msg):
  msg2 = gr.Textbox(value="",placeholder="Ask a question", container=True)
  return msg2


css_template="""

    .back{
        background-image: url('https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Large_scale_structure_of_dark_matter.tif/lossy-page1-640px-Large_scale_structure_of_dark_matter.tif.jpg');

    }
"""
def demo():
    with gr.Blocks(theme=gr.themes.Ocean(primary_hue="blue", secondary_hue="blue"),css=css_template,elem_classes="back") as demo:
        with gr.Column(elem_classes="back"):
          gr.HTML("""
          <center style="color:white;"><h1 style="color:white;">Simple RAG System</h1>
          This is a prototype application for Simple RAG Systtem. <br>The application information based on the dataset given.
          <center>
          """)
          state = gr.State([])

          with gr.Row():
              with gr.Column():
                  chatbot = gr.Chatbot()
                  with gr.Row():
                      msg = gr.Textbox(placeholder="Ask a question", container=True)
                  with gr.Row():
                      submit_btn = gr.Button("Submit")

          submit_btn.click(build_system2, \
            inputs=[msg,state], \
            outputs=[chatbot,state], \
            queue=True)

          submit_btn.click(clear, \
            inputs=[msg], \
            outputs=[msg], \
            queue=True)




    demo.queue().launch(debug=True)

if __name__ == "__main__":
    demo()

/tmp/ipython-input-2278498909.py:27: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://749a90727752055bc2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://749a90727752055bc2.gradio.live
